In [ ]:
# Upload the excel file
from google.colab import files
files.upload()

In [ ]:
import pandas as pd
import numpy as np

# Import the excel data

In [ ]:
# AC characteristics sheet
df_ac = pd.read_excel('Case study Vindiata.xlsx', sheet_name = 'AC characteristics', header = 3).sort_values(by=['Aircraft Type'])

# Drop irrelevant columns
df_ac.drop(df_ac.columns[0], axis = 1, inplace = True)
df_ac.drop(df_ac.iloc[:, 5:], inplace = True, axis = 1)
df_ac

,Aircraft Type,Range (Km),Ave. Speed (km/h),Number of Seats,Costs per flight hour
0,A320,5000,800,150,5000
1,A330,8000,900,250,7500
5,ATR72,1000,650,75,2750
2,B737,5000,800,150,5100
3,B747,10000,900,350,12000
4,Q400,1500,750,90,3500


In [ ]:
# Operations sheet
df_op = pd.read_excel('Case study Vindiata.xlsx', sheet_name = 'Operations', header = 3)
df_op.drop(df_op.columns[0], axis = 1, inplace = True)

# Rename columns that were subset of the header 'Hours flown'
new = ['Jan-14','Feb-14','Mar-14','Apr-14','May-14','Jun-14','Jul-14','Aug-14','Sep-14','Oct-14','Nov-14','Dec-14']
df_op.rename(columns=dict(zip(df_op.columns[2:], new)), inplace=True)

# Calculate total hours flown
df_op['Total hours flown'] = df_op.iloc[:,2:].sum(axis=1)

df_op.head()

,Aircraft Name,Aircraft Type,Jan-14,Feb-14,Mar-14,Apr-14,May-14,Jun-14,Jul-14,Aug-14,Sep-14,Oct-14,Nov-14,Dec-14,Total hours flown
0,A,A320,250,251,248,253,243,245,248,246,256,256,251,252,2999
1,B,A330,300,301,298,303,293,295,298,296,306,306,301,302,3599
2,C,B737,245,246,243,248,238,240,243,241,251,251,246,247,2939
3,D,B747,310,311,308,313,303,305,308,306,316,316,311,312,3719
4,E,Q400,200,201,198,203,193,195,198,196,206,206,201,202,2399


In [ ]:
# City pairs sheet
df_cp = pd.read_excel('Case study Vindiata.xlsx', sheet_name = 'City pairs', header = 3)

# Drop irrelevant columns
df_cp.drop(df_cp.columns[0], axis = 1, inplace = True)
df_cp.drop(df_cp.iloc[:, 4:], inplace = True, axis = 1)

# Remove unnecessary characters from column names
df_cp.rename(columns = {'Pass. Demand \n(per day)':'Pass. Demand (per day)'}, inplace = True)
df_cp

,Origin City,Desitnation City,Pass. Demand (per day),Distance (km)
0,AA,BB,420,3000
1,BB,CC,450,6500
2,CC,AA,300,400
3,AA,DD,300,1000


In [ ]:
# Create a copy dataframe for 'Operations' dataframe
df_op2 = df_op.copy()

# Calculate total hours flown for each aircraft type
df_op2 = df_op2.groupby(['Aircraft Type']).sum()

# Display both dataframes 'AC characteristics' & 'Operations'
df = df_ac.merge(df_op2, how='left', on='Aircraft Type')
df

,Aircraft Type,Range (Km),Ave. Speed (km/h),Number of Seats,Costs per flight hour,Jan-14,Feb-14,Mar-14,Apr-14,May-14,Jun-14,Jul-14,Aug-14,Sep-14,Oct-14,Nov-14,Dec-14,Total hours flown
0,A320,5000,800,150,5000,1240,1245,1230,1255,1205,1215,1230,1220,1270,1270,1245,1250,14875
1,A330,8000,900,250,7500,1820,1826,1808,1838,1778,1790,1808,1796,1856,1856,1826,1832,21834
2,ATR72,1000,650,75,2750,795,799,787,807,767,775,787,779,819,819,799,803,9536
3,B737,5000,800,150,5100,985,989,977,997,957,965,977,969,1009,1009,989,993,11816
4,B747,10000,900,350,12000,920,923,914,929,899,905,914,908,938,938,923,926,11037
5,Q400,1500,750,90,3500,790,794,782,802,762,770,782,774,814,814,794,798,9476


# Total cost by Aircraft Type
**Total cost = Total hours flown * Costs/hour**

In [ ]:
df['Total cost ($)'] = df['Total hours flown'] * df['Costs per flight hour']
df[['Aircraft Type', 'Total cost ($)']]

,Aircraft Type,Total cost ($)
0,A320,74375000
1,A330,163755000
2,ATR72,26224000
3,B737,60261600
4,B747,132444000
5,Q400,33166000


# Cost per seat per km by Aircraft Type
**Cost per seat per km = Total cost / (No. of seats * Total hours * Avg. speed)**

In [ ]:
df['$ Cost per seat per km'] = df['Total cost ($)'] / (df['Number of Seats'] * df['Total hours flown'] * df['Ave. Speed (km/h)'])
df[['Aircraft Type', '$ Cost per seat per km']].sort_values(by=['$ Cost per seat per km'])

,Aircraft Type,$ Cost per seat per km
1,A330,0.033333
4,B747,0.038095
0,A320,0.041667
3,B737,0.042500
5,Q400,0.051852
2,ATR72,0.056410


**Observation**: Aircraft Type **A330** has the lowest cost of **$0.033333** per seat per km flown

In [ ]:
# Drop columns not needed for evaluation
df.drop(df.iloc[:, 5:17], axis = 1, inplace = True)
df.drop(df.columns[[2,4,5,6]], axis = 1, inplace = True)
df

,Aircraft Type,Range (Km),Number of Seats,$ Cost per seat per km
0,A320,5000,150,0.041667
1,A330,8000,250,0.033333
2,ATR72,1000,75,0.056410
3,B737,5000,150,0.042500
4,B747,10000,350,0.038095
5,Q400,1500,90,0.051852


# Finding suitable airlines for travel

**Methodology for same aircraft type between a city pair:**

* Check if an aircraft's *range* if greater than the *distance* between the 2 cities under observation
* If the condition is satisfied, we calculate the cost of the trip as follows:

  **Total cost of the trip = Cost per seat per km * Number of Seats * Trip distance * Number of trips**;
  
  where, *Number of trips* = *Pass. Demand per day* / *Number of Seats*

In [ ]:
for a,b,c,d in df_cp.itertuples(index=False):
  df[str(a)+'-'+str(b)+' cost'] = np.where(df['Range (Km)'] >= d, (df['$ Cost per seat per km'] * df['Number of Seats'] * d * np.ceil(c / df['Number of Seats'])).astype(int), "NaN")

# Display the dataframe with added travel cost
df

,Aircraft Type,Range (Km),Number of Seats,$ Cost per seat per km,AA-BB cost,BB-CC cost,CC-AA cost,AA-DD cost
0,A320,5000,150,0.041667,56250,NaN,5000,12500
1,A330,8000,250,0.033333,50000,108333,6666,16666
2,ATR72,1000,75,0.056410,NaN,NaN,6769,16923
3,B737,5000,150,0.042500,57375,NaN,5100,12750
4,B747,10000,350,0.038095,80000,173333,5333,13333
5,Q400,1500,90,0.051852,NaN,NaN,7466,18666


**The following table summarizes the optimal aircraft type for each city pair along with the number of trips required:**

In [ ]:
ind = ['AA-BB','BB-CC','CC-AA','AA-DD'] # City pairs as indices
col = ['Optimal aircraft type','Number of trips required']

# No. of trips = Pass demand per day / No. of seats
data = [['A330', np.ceil(420/250)], ['A330', np.ceil(450/250)], ['A320', np.ceil(300/150)], ['A320', np.ceil(300/150)]]
res = pd.DataFrame(data, index = ind, columns = col)
res

,Optimal aircraft type,Number of trips required
AA-BB,A330,2.0
BB-CC,A330,2.0
CC-AA,A320,2.0
AA-DD,A320,2.0


**Methodology for a mixture of aircraft type between a city pair:**

In this case, the optimal aircraft types have to be found manually for each city pair.

---

**City Pair AA-BB**

In [ ]:
df2 = df.iloc[:,:4].copy()

# Travel cost and remaining passengers after 1st flight
df2['AA-BB cost 1'] = np.where(df2['Range (Km)'] >= 3000, (df2['$ Cost per seat per km'] * df2['Number of Seats'] * 3000).astype(int), 0)
df2['AA-BB remain 1'] = np.where(np.logical_and(420 - df2['Number of Seats'] >= 0, df2['AA-BB cost 1'] != 0), 420 - df2['Number of Seats'], 0)

# Total travel cost and remaining passengers after 2nd flight
df2['AA-BB cost 2'] = np.where(df2['Range (Km)'] >= 3000, (df2['$ Cost per seat per km'] * df2['Number of Seats'] * 3000).astype(int) + df2['AA-BB cost 1'], 0)
df2['AA-BB remain 2'] = np.where((df2['AA-BB remain 1'] - df2['Number of Seats']) >= 0, df2['AA-BB remain 1'] - df2['Number of Seats'], 0)
df2

,Aircraft Type,Range (Km),Number of Seats,$ Cost per seat per km,AA-BB cost 1,AA-BB remain 1,AA-BB cost 2,AA-BB remain 2
0,A320,5000,150,0.041667,18750,270,37500,120
1,A330,8000,250,0.033333,25000,170,50000,0
2,ATR72,1000,75,0.056410,0,0,0,0
3,B737,5000,150,0.042500,19125,270,38250,120
4,B747,10000,350,0.038095,40000,70,80000,0
5,Q400,1500,90,0.051852,0,0,0,0


Result for AA-BB: **2 aircrafts** of type **A330** are suitable.

---

**City Pair BB-CC**

In [ ]:
df3 = df.iloc[:,:4].copy()

# Travel cost and remaining passengers after 1st flight
df3['BB-CC cost 1'] = np.where(df3['Range (Km)'] >= 6500, (df3['$ Cost per seat per km'] * df3['Number of Seats'] * 6500).astype(int), 0)
df3['BB-CC remain 1'] = np.where(np.logical_and(450 - df3['Number of Seats'] >= 0, df3['BB-CC cost 1'] != 0), 450 - df3['Number of Seats'], 0)

# Total travel cost and remaining passengers after 2nd flight
df3['BB-CC cost 2'] = np.where(df3['Range (Km)'] >= 6500, (df3['$ Cost per seat per km'] * df3['Number of Seats'] * 6500).astype(int) + df3['BB-CC cost 1'], 0)
df3['BB-CC remain 2'] = np.where((df3['BB-CC remain 1'] - df3['Number of Seats']) >= 0, df3['BB-CC remain 1'] - df3['Number of Seats'], 0)
df3

,Aircraft Type,Range (Km),Number of Seats,$ Cost per seat per km,BB-CC cost 1,BB-CC remain 1,BB-CC cost 2,BB-CC remain 2
0,A320,5000,150,0.041667,0,0,0,0
1,A330,8000,250,0.033333,54166,200,108332,0
2,ATR72,1000,75,0.056410,0,0,0,0
3,B737,5000,150,0.042500,0,0,0,0
4,B747,10000,350,0.038095,86666,100,173332,0
5,Q400,1500,90,0.051852,0,0,0,0


Result for BB-CC: Since most of the aircrafts are incompatible for this journey, **2 aircrafts** of type **A330** are suitable.

---

**City Pair CC-AA**

In [ ]:
df4 = df.iloc[:,:4].copy()

# Travel cost and remaining passengers after 1st flight
df4['CC-AA cost 1'] = np.where(df4['Range (Km)'] >= 400, (df4['$ Cost per seat per km'] * df4['Number of Seats'] * 400).astype(int), 0)
df4['CC-AA remain 1'] = np.where(np.logical_and(300 - df4['Number of Seats'] >= 0, df4['CC-AA cost 1'] != 0), 300 - df4['Number of Seats'], 0)

# Total travel cost and remaining passengers after 2nd flight
df4['CC-AA cost 2'] = np.where(np.logical_and(df4['Range (Km)'] >= 400, df4['CC-AA remain 1'] != 0), (df4['$ Cost per seat per km'] * df4['Number of Seats'] * 400).astype(int) + df4['CC-AA cost 1'], df4['CC-AA cost 1'])
df4['CC-AA remain 2'] = np.where((df4['CC-AA remain 1'] - df4['Number of Seats']) >= 0, df4['CC-AA remain 1'] - df4['Number of Seats'], 0)
df4

,Aircraft Type,Range (Km),Number of Seats,$ Cost per seat per km,CC-AA cost 1,CC-AA remain 1,CC-AA cost 2,CC-AA remain 2
0,A320,5000,150,0.041667,2500,150,5000,0
1,A330,8000,250,0.033333,3333,50,6666,0
2,ATR72,1000,75,0.056410,1692,225,3384,150
3,B737,5000,150,0.042500,2550,150,5100,0
4,B747,10000,350,0.038095,5333,0,5333,0
5,Q400,1500,90,0.051852,1866,210,3732,120


Result for CC-AA: If 1 A330 and 1 ATR72 are used, the cost is still $25 more than using **2 aircrafts** of type **A320**. Hence, the latter is preferred.

---

**City Pair AA-DD**

In [ ]:
df5 = df.iloc[:,:4].copy()

# Travel cost and remaining passengers after 1st flight
df5['AA-DD cost 1'] = np.where(df5['Range (Km)'] >= 1000, (df5['$ Cost per seat per km'] * df5['Number of Seats'] * 1000).astype(int), 0)
df5['AA-DD remain 1'] = np.where(np.logical_and(300 - df5['Number of Seats'] >= 0, df5['AA-DD cost 1'] != 0), 300 - df5['Number of Seats'], 0)

# Total travel cost and remaining passengers after 2nd flight
df5['AA-DD cost 2'] = np.where(np.logical_and(df5['Range (Km)'] >= 1000, df5['AA-DD remain 1'] != 0), (df5['$ Cost per seat per km'] * df5['Number of Seats'] * 1000).astype(int) + df5['AA-DD cost 1'], df5['AA-DD cost 1'])
df5['AA-DD remain 2'] = np.where((df5['AA-DD remain 1'] - df5['Number of Seats']) >= 0, df5['AA-DD remain 1'] - df5['Number of Seats'], 0)
df5

,Aircraft Type,Range (Km),Number of Seats,$ Cost per seat per km,AA-DD cost 1,AA-DD remain 1,AA-DD cost 2,AA-DD remain 2
0,A320,5000,150,0.041667,6250,150,12500,0
1,A330,8000,250,0.033333,8333,50,16666,0
2,ATR72,1000,75,0.056410,4230,225,8460,150
3,B737,5000,150,0.042500,6375,150,12750,0
4,B747,10000,350,0.038095,13333,0,13333,0
5,Q400,1500,90,0.051852,4666,210,9332,120


Result for AA-DD: Similar to the previous case, **2 aircrafts** of type **A320** will be better than a mixture of different types.

---

Conclusion: **4 aircrafts of type A330 and 4 of type A320 are most optimal for airline B.**

Suggestion: For this problem statement. *SQL* is more suitable than *Python*.